# Fix units of emp:

Emp is the concentrating rate in m/s.  At one of the points where the model is going unstable (157, 32) it's 0.159 m/s. Emp might be in kg/m2/s --> to convert divide by rho.

In [31]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np

from salishsea_tools import (
    nc_tools,
    viz_tools,
)

%matplotlib inline

In [96]:
folder = '/ocean/brogalla/GEOTRACES/data/ANHA12/'
file1 = 'ANHA12-EXH006_5d_icemod_y2002m01d25.nc'

In [97]:
ice = nc.Dataset(folder+file1)

In [98]:
ice

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: GDT 1.3
    file_name: ANHA12-EXH006_y2002m01d25_icemod.nc
    production: OPA9.0 beta
    TimeStamp: 2016-APR-26 01:11:34 GMT-0600
    dimensions(sizes): x(1632), y(2400), deptht(1), time_counter(1)
    variables(dimensions): float32 nav_lon(y,x), float32 nav_lat(y,x), float32 deptht(deptht), float32 time_counter(time_counter), float32 isnowthi(time_counter,y,x), float32 iicethic(time_counter,y,x), float32 iiceprod(time_counter,y,x), float32 ileadfra(time_counter,y,x), float32 iicetemp(time_counter,y,x), float32 ioceflxb(time_counter,y,x), float32 iicevelu(time_counter,y,x), float32 iicevelv(time_counter,y,x), float32 isstempe(time_counter,y,x), float32 isssalin(time_counter,y,x), float32 iocetflx(time_counter,y,x), float32 iocesflx(time_counter,y,x), float32 iocwnsfl(time_counter,y,x), float32 iocesafl(time_counter,y,x), float32 iwinstru(time_counter,y,x), float32 iwinstrv(time_coun

In [99]:
emp = ice.variables['iicesflx']
emp = np.array(emp)

In [100]:
temp = ice.variables['isstempe']
sali = ice.variables['isssalin']
temp = np.array(temp)
sali = np.array(sali)

In [101]:
sn_ice = ice.variables['ileadfra']
sn_wnd = ice.variables['iwinstru']
sn_ice = np.array(sn_ice)
sn_wnd = np.array(sn_wnd)

Convert T and S to density:

In [102]:
def calculate_density(t, s):
    # function from salishsea psu_tools!
    """Calculates the density given temperature in deg C (t)
    and salinity in psu (s). The reference for this calculation is:
    Pond and Pickard, (1983). Introductory Dynamical Oceanography

    :arg t: temperature array in deg C
    :type t: numpy array

    :arg s: salinity array in psu
    :type s: numpy array

    :returns: the density as an array (rho) in kg/m^3
    """

    rho = (
        999.842594 + 6.793952e-2 * t
        - 9.095290e-3 * t*t + 1.001685e-4 * t*t*t
        - 1.120083e-6 * t*t*t*t + 6.536332e-9 * t*t*t*t*t
        + 8.24493e-1 * s - 4.0899e-3 * t*s
        + 7.6438e-5 * t*t*s - 8.2467e-7 * t*t*t*s
        + 5.3875e-9 * t*t*t*t*s - 5.72466e-3 * s**1.5
        + 1.0227e-4 * t*s**1.5 - 1.6546e-6 * t*t*s**1.5
        + 4.8314e-4 * s*s
        )

    return rho

In [103]:
rho = calculate_density(temp,sali)

In [104]:
print(rho.shape)
print(emp.shape)

(1, 2400, 1632)
(1, 2400, 1632)


In [105]:
print(np.amax(emp))
print(np.amin(emp))
print(np.average(emp))

326.873
0.0
30.7726


In [106]:
emp_fixed = np.divide(emp, rho)

In [107]:
print(np.amax(emp_fixed))
print(np.amin(emp_fixed))
print(np.average(emp_fixed))

0.319346
0.0
0.0300482


### Create NetCDF file:

In [108]:
# Load structure from sample dataset:
BX = ice.dimensions['x']
BY = ice.dimensions['y']

In [109]:
# Setup the new NetCDF file:
ncd = nc.Dataset('ANHA12-EXH006_5d_icemod_y2002m01d25-fixed.nc', 'w', zlib=True)
nc_tools.init_dataset_attrs(
    ncd,
    title='ANHA12 icemod',
    notebook_name='fix_emp_ice',
    nc_filepath='./.nc',
    comment='Emp adjusted ANHA12 mesh file')

ncd.createDimension('x',len(BX))
ncd.createDimension('y',len(BY))
ncd.createDimension('t',None)

file format: NETCDF4
Conventions: CF-1.6
title: ANHA12 icemod
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/ccar-modeling/analysis-birgit/src/tip/notebooks/ANHA12/fix_emp_ice.ipynb
references: REQUIRED
history: [2018-07-16 11:05:25] Created netCDF4 zlib=True dataset.
comment: Emp adjusted ANHA12 mesh file


<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 't', size = 0

In [110]:
iicesflx = ncd.createVariable('iicesflx', 'float32', ('t','y','x'))
iicesflx.long_name = 'Solar flux at ice/ocean surface'
iicesflx.units = 'm/s'
iicesflx[:] = emp_fixed

ileadfra = ncd.createVariable('ileadfra', 'float32', ('t','y','x'))
ileadfra.long_name = 'Ice concentration'
ileadfra.units = '%'
ileadfra[:] = sn_ice

iwinstru = ncd.createVariable('iwinstru', 'float32', ('t','y','x'))
iwinstru.long_name = 'Wind ice stress u'
iwinstru.units = 'Pa'
iwinstru[:] = sn_wnd

In [111]:
nc_tools.check_dataset_attrs(ncd)
ncd.close()

Missing value for dataset attribute: references
